In [ ]:
from osgeo import gdal
from math import floor
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## A Notebook used to tile an entire SAR image into distinct footprints to pass into the trained model

In [ ]:
# Opening the raster
raster = "./test-data/s1a-ew-grd-hv-20210304t163412-20210304t163512-036851-045567-002.tiff"
ds = gdal.Open(raster)

print("Projection: ", ds.GetProjection()) # get projection
print("Tiff width (pixels):", ds.RasterXSize)  # number of columns
print("Tiff Height (pixels):", ds.RasterYSize)  # number of rows
print("Band count:", ds.RasterCount)  # number of bands
print("GeoTransform Matrix", ds.GetGeoTransform())
print("Statistics:", ds.GetRasterBand(1).GetStatistics(0,1))

data_array2 = ds.GetRasterBand(1).ReadAsArray() # Indexing for raster bands starts at 1, not 0

plt.figure(figsize=(10, 10))
plt.imshow(data_array2) # Data array contains elevation data from the raster band. Is this always true?
plt.colorbar()
plt.show()

In [ ]:
print("Tiff width (pixels):", ds.RasterXSize)  # number of columns
print("Tiff Height (pixels):", ds.RasterYSize)  # number of rows
FOOTPRINT_WIDTH = 17
VISUAL_FACTOR = 10

x_corners = []
y_corners = []

tile_num = 0
y_index = 0
for i in range(0, ds.RasterYSize, FOOTPRINT_WIDTH):
    x_index = 0
    y_index += 1
    # If there isn't enough data on the bottom edge, break
    if ds.RasterYSize - i < FOOTPRINT_WIDTH:
      break    
    for j in range(0, ds.RasterXSize, FOOTPRINT_WIDTH):
        x_index += 1
        # If there isn't enough data on the right edge, continue to the next row
        if ds.RasterXSize - j < FOOTPRINT_WIDTH:
            continue
        # If it's the 10th tile, add it to the array
        if x_index % VISUAL_FACTOR == 0 and y_index % VISUAL_FACTOR == 0:
            x_corners.append(j)
            y_corners.append(i)
        # Tile the image
            window = (i, j, FOOTPRINT_WIDTH, FOOTPRINT_WIDTH)
        # gdal.Translate(tile_num, ds2, srcWin = window)
        tile_num += 1
        print(i , j)

plt.figure(figsize=(10, 10))
plt.imshow(data_array2) # Data array contains elevation data from the raster band. Is this always true?
plt.scatter(x=x_corners, y=y_corners, c='r', s=1, alpha=0.5)
plt.title(f"SAR Tiling for Trained Model (Factor of {VISUAL_FACTOR})")
plt.figtext(0.9, 0.05, f'Total of {tile_num} tiles', horizontalalignment='right')
plt.show()